In [ ]:
from tensorflow import keras
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True,validation_split=0.2)

In [ ]:
train = train_datagen.flow_from_directory('Dataset',target_size = (224,224), batch_size = 32,class_mode = 'categorical',subset = 'training')

In [ ]:
test = train_datagen.flow_from_directory('Dataset',target_size = (224,224), batch_size = 32, class_mode = 'categorical',subset = 'validation')

In [ ]:
from keras.applications import ResNet50

conv_base = ResNet50(weights = "imagenet",
include_top = False,
input_shape = (224, 224, 3))
conv_base.summary()

In [ ]:
import numpy as np
def extractfeatures(sample_count, generator):
    i = 0
    batch_size = 32
    features = np.zeros(shape=(sample_count, 7, 7, 2048))
    labels = np.zeros(shape=(sample_count, 3))
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i+1) * batch_size] = features_batch
        labels[i * batch_size: (i+1) * batch_size] = labels_batch
        i += 1
        if ((i * batch_size % 1000) == 0 ):
            print("processed size =", i * batch_size)
        if i * batch_size >= sample_count:
            break
    return features, labels

In [ ]:
train_features, train_labels = extractfeatures(85902, train)
test_features, test_labels = extractfeatures(21406, test)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape = (7, 7, 2048)), 
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5)
    keras.layers.Dense(158, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_features, train_labels, steps_per_epoch = 85902 // 32, epochs = 10)